In [ ]:
import pandas as pd
import numpy as np

#Import datasheet
df = pd.read_csv('/home/sanda/Desktop/Thesis/Datasheet/csv file 6/train_dataframes.csv')
df.head()

df['Date Time'] = pd.to_datetime(df['datetime'], yearfirst=True)
df['Year'] = df['Date Time'].dt.year
df['Month'] = df['Date Time'].dt.month
df['Day'] = df['Date Time'].dt.day
df['Hour'] = df['Date Time'].dt.hour
df['Minute'] = df['Date Time'].dt.minute
df['Second'] = df['Date Time'].dt.second
df.head()

df['Demand'] = pd.to_numeric(df['DEMAND'], errors='coerce')

df.drop(['datetime'], axis='columns', inplace=True, errors='ignore')
df.drop(['week_X-2'], axis='columns', inplace=True, errors='ignore')
df.drop(['week_X-3'], axis='columns', inplace=True, errors='ignore')
df.drop(['week_X-4'], axis='columns', inplace=True, errors='ignore')
df.drop(['MA_X-4'], axis='columns', inplace=True, errors='ignore')
df.drop(['dayOfWeek'], axis='columns', inplace=True, errors='ignore')
df.drop(['weekend'], axis='columns', inplace=True, errors='ignore')
df.drop(['holiday'], axis='columns', inplace=True, errors='ignore')
df.drop(['Holiday_ID'], axis='columns', inplace=True, errors='ignore')
df.drop(['hourOfDay'], axis='columns', inplace=True, errors='ignore')
df.drop(['T2M_toc'], axis='columns', inplace=True, errors='ignore')
df.drop(['DEMAND'], axis='columns', inplace=True, errors='ignore')
df.drop(['Date Time'], axis='columns', inplace=True, errors='ignore')
df.drop(['Minute'], axis='columns', inplace=True, errors='ignore')
df.drop(['Second'], axis='columns', inplace=True, errors='ignore')
print(df.dtypes)
df.head

#Transform the Data
#Into array
X0 = []
X1 = []
X2 = []
X3 = []
X4 = []
y = []

#Append each of the values into this empty list
for i in range(0, df.shape[0]-24):
    X0.append(df.iloc[i:i+24, 0])
    X1.append(df.iloc[i:i+24, 1])
    X2.append(df.iloc[i:i+24, 2])
    X3.append(df.iloc[i:i+24, 3])
    X4.append(df.iloc[i:i+24, 4])
    y.append(df.iloc[i+24, 4])

X0, X1, X2, X3, X4, y = np.array(X0), np.array(X1), np.array(X2), np.array(X3), np.array(X4), np.array(y)
y.shape

y = np.reshape(y, (len(y),1))
y.shape

#Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X0 = scaler.fit_transform(X0) 
X1 = scaler.fit_transform(X1) 
X2 = scaler.fit_transform(X2) 
X3 = scaler.fit_transform(X3) 
X4 = scaler.fit_transform(X4) 
y = scaler.fit_transform(y) 
X4

X = np.stack([X0, X1, X2, X3, X4], axis=2)
X.shape

In [ ]:
#Build the LSTM model
X_train, X_test = X[ :-720], X[-720: ] 
y_train, y_test = y[ :-720], y[-720: ]
X_train.shape[1], X_train.shape[2]

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import optimizers

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.summary()

#Early stop the training if validation problem goes up
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
filepath = 'models/{epoch:02d}-{loss:.4f}-{val_loss:.4f}-{mae:.4f}-{val_mae:.4f}.hdf5'
callbacks = [EarlyStopping(monitor='val_loss', patience=20), ModelCheckpoint(filepath, monitor='loss', save_best_only=True, model='min')]

#Training the model
optimizers.SGD(momentum=0.9)
model.compile(optimizer='SGD', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=5, callbacks=callbacks, batch_size=16)

In [ ]:
#Evaluate and prediction
MSE, MAE = model.evaluate(X_test, y_test)
print(MSE, MAE)

predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions

#Plot
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.figure(figsize=(24,12))
plt.title('1 hour ahead Electricity Demand Forecasting')
plt.xlabel('Step(1 hour)', fontsize=14)
plt.ylabel('Electricity Demand', fontsize=18)
plt.plot(predictions)
plt.plot(scaler.inverse_transform(y_test))
plt.legend(['Forecast', 'Actual'])
plt.show()